In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import random
import csv
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

from solar_module import SolarModule, generate_shading, generate_gaussian
from solar_cell import all_series_bypass, SP_interconnection, TCT_interconnection
from string_to_embedding import string_to_embedding, super_to_embedding

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Model(nn.Module):
    def __init__(self, imgchannels=1, fdim=8, zdim=32, mdim=4096, kern=3, imgshape=(3,3), stride=1, pad=1):
        
        super(Model, self).__init__()
        
        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 8 convolutional features, with a square kernel size of 2
        self.conv1 = nn.Conv2d(imgchannels, fdim, kern, stride, pad) # (1, 8, 3, 1)

        # Second 2D convolutional layer, taking in the 8 input layers,
        # outputting 32 convolutional features, with a square kernel size of 2
        self.conv2 = nn.Conv2d(fdim, zdim, kern, stride, pad) # (8, 32, 3, 1, 1)

        # Designed to ensure that adjacent pixels are either all 0s or all active
        # with an input probability
        #self.dropout1 = nn.Dropout2d(0.25)
        #self.dropout2 = nn.Dropout2d(0.5)
        
        ldim = zdim*imgshape[0]*imgshape[1]
        self.ldim = ldim

        # First fully connected layer
        self.fc1 = nn.Linear(ldim, mdim) #add a middle linear layer, with size 
        self.fc3 = nn.Linear(mdim, mdim)
        outdim = (imgshape[0]*imgshape[1])**2*3+imgshape[0]*imgshape[1]*2
        #outdim = (imgshape[0]*imgshape[1])**2*3
        self.fc2 = nn.Linear(mdim, outdim)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        #print(x.shape)
        
        x = self.conv2(x)
        x = F.relu(x)
        #print(x.shape)
        
        #x = self.dropout1(x)
        x = x.view(-1, self.ldim)
        #print(x.shape)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        #print(x.shape)
        #x = self.dropout2(x)
        x = self.fc2(x)
        #print(x.shape)
        
        x = F.relu(x)
        #x = x.bool()
        
        return x

model = Model(imgshape=(10, 6)).to(device)
print(model)
print(device)
for param in model.parameters():
  print(param.data)

In [ ]:
rand_smap = generate_gaussian(10, 10, 6)
plt.imshow(rand_smap)

In [ ]:
rand_smap = torch.Tensor(np.expand_dims(rand_smap,0))
print(rand_smap)
print(rand_smap.shape)

In [ ]:
result = model(rand_smap)
print(result)
print(result.shape)

In [ ]:
""" Initialize Hyperparameters """
#10,000 iterations, batching of 32

#1e-3 learning rate

# vary each training parameter individually, run multiple experiments. Checkpoint each model, filename w/ parameters

batch_size = 32 # 32-64 is advisable
learning_rate = 1e-3 # sensitivity of gradient descent
num_epochs = 1 # iterations over entire dataset 

In [ ]:
""" Generate 10x6 Shading Maps"""
filename = '10x6shading_maps.csv'
maps = []
for x in range(0, 1000):
    rand_smap = generate_gaussian(10, 10, 6)
    maps.append(rand_smap)

maps = pd.Series(maps, name='Shading Maps')
maps.to_csv(filename)

In [ ]:
""" Load Shading Maps"""
read_in = pd.read_csv('10x6shading_maps.csv')
    
def convert_to_array(string):
    a = np.matrix(string).reshape(10, 6)
    a = np.array(a)
    return a
shading_series = [convert_to_array(s) for s in read_in['Shading Maps']]
data = [torch.Tensor(np.expand_dims(s,0)) for s in shading_series]
train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
""" Create custom dataset """
class CustomDataset(Dataset):
    def __init__(self, df, rotate=None):
        self.df = df
        self.rotate = rotate # for rotation of shading maps?

    def __getitem__(self, index):
        #shading_map = self.df.iloc[index, 0]
        shading_map = shading_series[index]
        #print(shading_map)
        shading_map = torch.Tensor(shading_map)/10
        shading_map = shading_map.unsqueeze(0)
        
        return shading_map
        
    def __len__(self):
        return len(df)

dataset = CustomDataset(train_loader)
print(dataset.__getitem__(2))
print(dataset.__getitem__(2)[0].shape)
print(len(train_loader))

In [ ]:
model = Model(imgshape=(10, 6)).to(device)
print(model)
print(device)

In [ ]:
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
ratio_list = []
self_connections = []
conflicting_connections = []
loss_list = []

In [ ]:
# iterate over epochs
for epoch in range(num_epochs):
    
    # iterate over training dataset by batch
    for i, data in enumerate(train_loader, 0):

            shading_map = data
            shading_map = shading_map.to(device)
            result = model(shading_map)
            
            average_ratio = 0
            self_c = 0
            conflicting_c = 0
            for j in range(0, batch_size):
                embedding = result[j]
                embedding = np.squeeze(embedding.detach())
                embedding, terminal_array = embedding[:10800], embedding[10800:]
                embedding = embedding.reshape(10, 6, 10, 6, 3)
                terminal_array = terminal_array.reshape(10, 6, 2)
                
                series_connections = torch.count_nonzero(embedding[...,0]) + torch.count_nonzero(embedding[...,1])
                parallel_connections = torch.count_nonzero(embedding[...,2])
                if parallel_connections == 0:
                    average_ratio += 0
                elif parallel_connections > 0:
                    average_ratio += (series_connections / parallel_connections)
                
                moduleobj = SolarModule(10, 6)
                moduleobj.embedding = np.array(embedding.numpy(), dtype=bool)
                moduleobj.terminal_array = np.array(terminal_array.numpy(), dtype=bool)
                filtering = moduleobj.filter_embedding()
                self_c += filtering[0]
                conflicting_c += filtering[1]
                
            average_ratio /= batch_size
            self_c /= batch_size
            conflicting_c /= batch_size
            ratio_list.append(float(average_ratio))
            self_connections.append(self_c)
            conflicting_connections.append(conflicting_c)
            
            #loss = torch.mean(entropy_loss(result, embeddings),axis=1)
            #loss = torch.mean(loss*(1-power))
            
            loss = torch.tensor(conflicting_c, requires_grad=True)
            
            loss_list.append(float(loss))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            print(i, ratio_list[i], self_connections[i], conflicting_connections[i], loss)
    
    print('Epoch {}: Loss {}'.format(epoch, loss))
    
# create validation/evaluation batch of 10 or so shading maps, calculate the power from that
# visualise embeddings or circuit diagram against shading map

# use embeddings generated by superstrings for supervised approach. 
# pre-train on training_data.csv
# binary cross entropy between model and training data

# sort training_data.csv by performance (power)
# train same model by binary cross entropy
# use entire dataset, then use higher performing subset, then a subset of that, etc. 
# label = embedding


In [ ]:
for x in loss_graph:
    print(x)

In [ ]:
''' save model checkpoint '''

# set model checkpoint path inc. hyperparams
_path = 'unsupervised-f{}-k{}-z{}-i{}-v{}.pth'.format(8,3,256,115,'01')
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
}, _path)


In [ ]:
""" load model checkpoint """
''' load model checkpoint from file '''

# init model and optimiser
#model = TheModelClass(*args, **kwargs)
#optimizer = TheOptimizerClass(*args, **kwargs)

#_path = '../data/model/cnn-vae-f16-k5-z128-i121-v01.pth'
#_path = '../data/model/cnn-vae-f16-k3-z128-i115-v01.pth'
_path = 'Checkpoints/Batch42FromInit.pth'

# load checkpoint
checkpoint = torch.load(_path)

# update model
#model.load_state_dict(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'])

# update optimiser
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# set epoch and loss
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# set model train/eval state
#model.eval()
#model.train()